## 📚 Prerequisites

Ensure that your Azure Services are properly set up, your Conda environment is created, and your environment variables are configured as per the instructions in the [README.md](README.md) file.


In [1]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-elearning-fusion-content"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-elearning-fusion-content


In [3]:
!pip install aiofiles asyncio

: 

In [17]:
from src.ocr.transformer import GPT4VisionManager
gpt_vision_client = GPT4VisionManager(deployment_name="vision-gpt4")

In [3]:
# Insert your video SAS URL, e.g. https://<your-storage-account-name>.blob.core.windows.net/<your-container-name>/<your-video-name>?<SAS-token>
video_SAS_url: str = "https://testeastusdev001.blob.core.windows.net/ocrtest/video_test.mp4"
# This index name must be unique and contain no white spaces.
# It must start with alphanumeric, can contain hyphens but they must be followed 
# by alphanumeric (no consecutive hyphens or trailing hyphen).
# It must be 24 characters or less.
VIDEO_INDEX_NAME = "Unique-Index-Name"
VIDEO_DOCUMENT_ID = "AOAIChatDocument"

In [4]:
gpt_vision_client.video_indexer.process_video_indexing(VIDEO_INDEX_NAME, 
                                                       video_SAS_url, 
                                                       VIDEO_DOCUMENT_ID)


2024-05-17 00:24:35,263 - micro - MainProcess - ERROR    Failed to create video index 'Unique-Index-Name': 409 Client Error: Conflict for url: https://swizertlandnorth-cv-s1.cognitiveservices.azure.com/computervision/retrieval/indexes/Unique-Index-Name?api-version=2023-05-01-preview (transformer.py:create_video_index:449)
2024-05-17 00:24:35,265 - micro - MainProcess - ERROR    Error during video indexing process: 409 Client Error: Conflict for url: https://swizertlandnorth-cv-s1.cognitiveservices.azure.com/computervision/retrieval/indexes/Unique-Index-Name?api-version=2023-05-01-preview (transformer.py:process_video_indexing:526)


In [5]:
import re

In [6]:
# System messages and user prompt
sys_message = """
Your task is to assist in analyzing and optimizing creative assets. 
You will be presented with advertisement videos for products. 
First describe the video in detail paying close attention to Product characteristics highlighted, 
Background images, Lighting, Color Palette and Human characteristics for persons in the video. 
Finally provide a summary of the video and talk about the main message the advertisement video tries to convey to the viewer. 
"""
user_prompt = "Summarize the ad video"

# Make sure that the content of type acv_document_id is first in the use content list like in this example.
# Otherwise unexpected behavior can happen.
messages = [
    {"role": "system", "content": [{"type": "text", "text": sys_message}]},
    {
        "role": "user",
        "content": [{"type": "acv_document_id", "acv_document_id": VIDEO_DOCUMENT_ID}, {"type": "text", "text": user_prompt}],
    },  # Prompt for the user
]


video_config = {
    "video_SAS_url": video_SAS_url,
    "video_index_name": VIDEO_INDEX_NAME,
}

# Call GPT-4 Turbo with Vision API and print the response
try:
    response = gpt_vision_client.call_GPT4V_video(messages,video_index=video_config)
    text = response["choices"][0]["message"]["content"]
    sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", text)
    for sentence in sentences:  # Print the content of the response
        print(sentence)
except Exception as e:
    print(f"Failed to call GPT-4 Turbo with Vision API. Error: {e}")

2024-05-17 00:24:41,382 - micro - MainProcess - INFO     Failed to make the request. Error: 500 Server Error: Internal Server Error for url: https://ml-workspace-dev-switzerlandnorth-001-aoai.openai.azure.com/openai/deployments/vision-gpt4/extensions/chat/completions?api-version=2023-07-01-preview (transformer.py:call_GPT4V_video:415)


Failed to call GPT-4 Turbo with Vision API. Error: 'choices'


In [2]:
import os
import time
import requests

# Configuration
GPT_4V_ENDPOINT = "https://ml-workspace-dev-switzerlandnorth-001-aoai.openai.azure.com/openai/deployments/vision-gpt4/extensions/chat/completions?api-version=2023-07-01-preview"
GPT_4V_KEY = "e59d6e551f504ad3bcacd27cb4b9b9bb"
VISION_API_ENDPOINT = "https://swizertlandnorth-cv-s1.cognitiveservices.azure.com/computervision"
VISION_API_KEY = "d33c395f239e4f03b847805c78a740f7"

## ingest the video
VIDEO_FILE_SAS_URL = "https://testeastusdev001.blob.core.windows.net/ocrtest/video_test.mp4"
VIDEO_INDEX_NAME = "test-index-lala"
VIDEO_DOCUMENT_ID = "test-document-id"


In [3]:
def create_video_index(vision_api_endpoint, vision_api_key, index_name):
    url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
    headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
    data = {
        "features": [
            {"name": "vision", "domain": "surveillance"}
        ]
    }
    response = requests.put(url, headers=headers, json=data)
    return response

def add_video_to_index(vision_api_endpoint, vision_api_key, index_name, video_url, video_id):
    url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions/my-ingestion?api-version=2023-05-01-preview"
    headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
    data = {
        'videos': [{'mode': 'add', 'documentId': video_id, 'documentUrl': video_url}]
    }
    response = requests.put(url, headers=headers, json=data)
    return response

def wait_for_ingestion_completion(vision_api_endpoint, vision_api_key, index_name, max_retries=30):
    url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions?api-version=2023-05-01-preview"
    headers = {"Ocp-Apim-Subscription-Key": vision_api_key}
    retries = 0
    while retries < max_retries:
        time.sleep(10)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            state_data = response.json()
            if state_data['value'][0]['state'] == 'Completed':
                print(state_data)
                print('Ingestion completed.')
                return True
            elif state_data['value'][0]['state'] == 'Failed':
                print(state_data)
                print('Ingestion failed.')
                return False
        retries += 1
    return False




In [4]:
# Step 1: Create an Index
response = create_video_index(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME)
print(response.status_code, response.text)

404 {"error":{"code":"404","message": "Resource not found"}}


In [5]:
# Step 2: Add a video file to the index
response = add_video_to_index(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME, VIDEO_FILE_SAS_URL, VIDEO_DOCUMENT_ID)
print(response.status_code, response.text)

404 {"error":{"code":"404","message": "Resource not found"}}


In [7]:

# Step 3: Wait for ingestion to complete
if not wait_for_ingestion_completion(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME, max_retries=10):
    print("Ingestion did not complete within the expected time.")

Ingestion did not complete within the expected time.


In [ ]:
## Chat with GPT-4V

headers = {
    "Content-Type": "application/json",
    "api-key": GPT_4V_KEY,
}

# Payload for the request
payload = {
    "dataSources": [
        {
            "type": "AzureComputerVisionVideoIndex",
            "parameters": {
                "computerVisionBaseUrl": f"{VISION_API_ENDPOINT}/computervision",
                "computerVisionApiKey": VISION_API_KEY,
                "indexName": VIDEO_INDEX_NAME,
                "videoUrls": [VIDEO_FILE_SAS_URL]
            }
        }
    ],
    "enhancements": {
        "video": {
            "enabled": True
        }
    },
    "messages": [
     {
          "role": "system",
          "content": [
               {
                    "type": "text",
                    "text": "You are an AI assistant that helps people find information."
               }
          ]
     },
     {
          "role": "user",
          "content": [
               {
                    "type": "acv_document_id",
                    "acv_document_id": "AOAIChatDocument"
               },
               {
                    "type": "text",
                    "text": "\n \nPlease describe video as detailed as possible - conetx is vscode demo for showcasing how copiot x works "
               }
          ]
     },
     {
          "role": "assistant",
          "content": [
               {
                    "type": "text",
                    "text": "The video appears to be a demonstration of how Copilot X functions within the Visual Studio Code (VSCode) editor. Copilot X is likely an AI-powered coding assistant that helps programmers write code more efficiently by providing suggestions and completing code snippets. The frames provided show the progression of a coding session in VSCode, focusing on a single screen with the editor open.\n\nTimestamp 00:00:00 to 00:00:33:\nThe initial frames show a VSCode window with multiple files and directories in the Explorer pane on the left side. The main editor pane is open to a file with JavaScript code. The code consists of import statements, a functional component definition, and the beginnings of a React component using JSX syntax. The cursor is active within the code, and the user appears to be either typing or receiving suggestions for code completion.\n\nTimestamp 00:00:37 to 00:01:29:\nAs the video progresses, we see the code evolving with more functions and JSX being added. The AI assistance seems to be suggesting code snippets and possibly auto-completing repetitive tasks or patterns recognized by the AI. The suggestions might include defining component states, handling events, or constructing render methods. Some frames show highlighted blocks of code, indicating that the AI has suggested a larger snippet that the user can accept or modify.\n\nThroughout the video, the user is interacting with the code—potentially accepting, modifying, or typing new code. The codebase grows, indicating the development of a more complex React component or application. The suggestions by Copilot X seem to be context-aware, providing assistance based on the existing code and possibly comments or other cues within the editor.\n\nThe demonstration likely showcases the features of Copilot X, such as:\n- Suggesting code snippets based on the context of the current codebase\n- Auto-completing repetitive coding patterns\n- Providing intelligent suggestions for imports, functions, and other coding constructs\n- Speeding up the development process by reducing the amount of manual coding required\n\nSince the provided frames only display the VSCode interface and do not include any audio or visible interaction from the user, the specific actions taken by the user (like accepting suggestions or typing) and the full capabilities of Copilot X are not explicitly demonstrated. However, the evolving state of the code across the frames suggests that the AI tool is actively being used to assist in coding tasks."
               }
          ]
     }
],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 800
}

# Send request
try:
    response = requests.post(GPT_4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

In [4]:
from src.ocr.document_intelligence import AzureDocumentIntelligenceManager

document_intelligence_client = AzureDocumentIntelligenceManager()

In [5]:
from src.extractors.blob_data_extractor import AzureBlobDataExtractor

blob_data_extractor = AzureBlobDataExtractor(container_name="ocrtest2")

In [6]:
url = blob_data_extractor.upload_file_to_blob("utils\\data\\copilot-productivity.pdf", "copilot-productivity.pdf")

2024-05-17 07:49:45,041 - micro - MainProcess - INFO     File utils\data\copilot-productivity.pdf uploaded to blob storage as copilot-productivity.pdf. (blob_data_extractor.py:upload_file_to_blob:214)


In [7]:
url

'https://testeastusdev001.blob.core.windows.net/ocrtest2/copilot-productivity.pdf'

In [8]:
model_type = "prebuilt-layout"
result_ocr = document_intelligence_client.analyze_document(
    document_input=url,
    model_type=model_type,
    output_format="markdown",
    features=["OCR_HIGH_RESOLUTION"]
)

2024-05-17 07:49:48,994 - micro - MainProcess - INFO     Blob URL detected. Extracting content. (document_intelligence.py:analyze_document:148)
2024-05-17 07:49:49,973 - micro - MainProcess - INFO     Successfully downloaded blob file copilot-productivity.pdf (blob_data_extractor.py:extract_content:88)


In [9]:
result_ocr.content

'Copilot/ChatGPT for productivity FAQ\n===\n\n\n## Table of Contents\n\n\n## Copilot/ChatGPT for productivity FAQ 1\n\n||||\n| - | - | - |\n| How to install || 1 |\n| Why copilot completion so slow ?. || 4 |\n| How can I get next suggestion || 5 |\n| | Sometimes the copilot suggests the wrong function, not the one I was expecting | 5 |\n| Some auto fill cases || 8 |\n| How to write unit test: || 9 |\n| How to explain code: || 10 |\n| How to translate code || 11 |\n| How to review code: || 14 |\n| | How to edit Jupiter notebook? (@Data Scientist) | 14 |\n| Copilot short cut || 17 |\n| How to install Copilot Chat (in Copilot X) :selected: || 17 |\n\nHow to install Copilot organization | Microsoft Open Source Management\n\n<!-- PageFooter="1\\. accept the guidelines:" -->\n\n<!-- PageNumber="0" -->\n\n| Review guidelines \n|| || · Of |\n| - | - |\n| Guidelines for GitHub Copilot at Microsoft | · Ge · GE |\n| You are responsible for the code you write with Copilot. All teams using Copilot 

You are an expert in [field/topic]. Write an outline for a PowerPoint presentation covering the following [topics]. Make it [number] slides.

Expand on each of the subtopics you provided earlier. You can consider elaborating on the key ideas, offering supporting examples, and explaining any details that you think would enhance the audience's understanding of the topic.

Expand on the ideas in bullet format with summary details. 

Write me VBA PowerPoint codes on [topic]. Make it [number] slides.

Suggest any images to be included in the slides to enhance the visual appeal. Provide also the Bing Image Creator prompts for the images you suggested


In [10]:
from src.aoai.azure_openai import AzureOpenAIManager

az = AzureOpenAIManager()

In [11]:
user_inputs = "write the document about how to use Copilot X for productivity, make it very technical, based on the document provided, and as extensive as possible."
topic = "Copilot X for Productivity Enhancement"
max_tokens = 3000
query = f'''Given the content extracted from various documents using Optical Character Recognition (OCR) technology and provided in markdown format, your task is to create a high-quality, detailed "How-To" guide. The guide should distill complex topics into accessible, step-by-step instructions tailored for users seeking to understand or implement specific processes or concepts.

userinputs: {user_inputs}
context: {result_ocr.content}

Essential Steps for Crafting the Guide:

1. **Content Synthesis**: Begin by synthesizing the OCR-extracted content. Identify crucial themes, technical concepts, and actionable instructions relevant to Copilot X and productivity enhancement. This synthesis forms the foundation of your guide's structure and content focus.

2. **Target Audience Clarification**: Clearly define the guide's target audience. Understanding the audience's technical background, familiarity with Copilot X, and productivity goals is essential for customizing the guide's complexity and instructional style.

3. **Structured Outline Development**: Construct a structured outline to organize the guide into coherent sections and subsections. Each section should concentrate on distinct aspects of using Copilot X for productivity, ensuring a logical progression from introductory concepts to advanced applications.

4. **Guide Composition**:
    a. **Introduction**: Craft an introduction that outlines the guide's objectives, the significance of Copilot X for productivity, and what the readers will gain.
    b. **Detailed Instructions**: Following the outline, elaborate on each section with clear, technical instructions. Incorporate step-by-step processes, code snippets, examples, and best practices specific to Copilot X.
    c. **Conclusion**: Summarize the key takeaways, suggest further reading or resources, and encourage steps for practical application.

5. **Comprehensive Review and Enhancement**: Thoroughly review the guide to ensure technical accuracy, clarity, and completeness. Revise any sections as necessary, and consider peer or expert feedback for additional insights.

6. **Final Formatting and Release**: Apply professional formatting to enhance readability and visual appeal. Use diagrams, screenshots, or videos where applicable. Publish the guide in a format accessible to your target audience, ensuring it's ready for distribution and application.

Additional Guidelines:

- Begin with a clear agenda and systematically develop content within designated sections.
- Employ straightforward language while explaining technical details, using examples to demystify complex concepts.
- Dedicate ample time to crafting high-quality content, prioritizing accuracy and user engagement.
- Base the guide explicitly on the OCR content and the nuanced requirements of the user's query regarding {topic}.
- minimun length of the document should be {max_tokens} tokens'''

In [14]:
response_content = az.generate_chat_response(conversation_history=[],
                          system_message_content='''You are tasked with creating detailed, user-friendly documentation based on multiple documents and complex topics. 
                          The goal is to distill this information into an easy-to-follow "How-To" guide. This documentation should 
                          be structured with clear headings, subheadings, and step-by-step instructions that guide the user through 
                          the necessary processes or concepts. Each section should be well-organized and written in simple language 
                          to ensure that the content is accessible and understandable to users with varying levels of expertise. 
                          The documentation should cover the setup, configuration, and usage of tools or techniques, 
                          including practical examples and troubleshooting tips to address common issues or challenges that users 
                          might encounter.''', 
                          query=query, 
                          max_tokens = 3000)
                        

2024-05-17 07:52:37,356 - micro - MainProcess - INFO     Sending request to OpenAI with query: Given the content extracted from various documents using Optical Character Recognition (OCR) technology and provided in markdown format, your task is to create a high-quality, detailed "How-To" guide. The guide should distill complex topics into accessible, step-by-step instructions tailored for users seeking to understand or implement specific processes or concepts.

userinputs: write the document about how to use Copilot X for productivity, make it very technical, based on the document provided, and as extensive as possible.
context: Copilot/ChatGPT for productivity FAQ
===


## Table of Contents


## Copilot/ChatGPT for productivity FAQ 1

||||
| - | - | - |
| How to install || 1 |
| Why copilot completion so slow ?. || 4 |
| How can I get next suggestion || 5 |
| | Sometimes the copilot suggests the wrong function, not the one I was expecting | 5 |
| Some auto fill cases || 8 |
| How to w

2024-05-17 07:53:55,944 - micro - MainProcess - INFO     Received response from OpenAI: # How-To Guide: Maximizing Productivity with Copilot X

## Introduction

Welcome to the comprehensive guide on utilizing Copilot X to enhance your productivity. This guide is designed for users who aim to leverage Copilot X's robust features to streamline their coding process, accelerate development timelines, and improve code quality. Whether you're a beginner or an experienced developer, this guide will provide you with detailed, step-by-step instructions and tips to effectively use Copilot X in your projects.

## Table of Contents

1. [Getting Started with Copilot X](#getting-started-with-copilot-x)
   - [Installation](#installation)
   - [Configuration](#configuration)
2. [Using Copilot X for Code Suggestions](#using-copilot-x-for-code-suggestions)
   - [Triggering Suggestions](#triggering-suggestions)
   - [Navigating Suggestions](#navigating-suggestions)
3. [Advanced Features](#advanced-featur

In [22]:
print(response_content)

# How-To Guide: Maximizing Productivity with Copilot X

## Introduction

Welcome to the comprehensive guide on utilizing Copilot X to enhance your productivity. This guide is designed for users who aim to leverage Copilot X's robust features to streamline their coding process, accelerate development timelines, and improve code quality. Whether you're a beginner or an experienced developer, this guide will provide you with detailed, step-by-step instructions and tips to effectively use Copilot X in your projects.

## Table of Contents

1. [Getting Started with Copilot X](#getting-started-with-copilot-x)
   - [Installation](#installation)
   - [Configuration](#configuration)
2. [Using Copilot X for Code Suggestions](#using-copilot-x-for-code-suggestions)
   - [Triggering Suggestions](#triggering-suggestions)
   - [Navigating Suggestions](#navigating-suggestions)
3. [Advanced Features](#advanced-features)
   - [Unit Testing with Copilot X](#unit-testing-with-copilot-x)
   - [Code Explanat

In [26]:
from docx import Document

def markdown_to_docx(markdown_text):
    doc = Document()
    lines = markdown_text.split('\n')
    
    for line in lines:
        if line.startswith('# '):  # Heading 1
            doc.add_heading(line[2:], level=1)
        elif line.startswith('## '):  # Heading 2
            doc.add_heading(line[3:], level=2)
        elif line.startswith('### '):  # Heading 3
            doc.add_heading(line[4:], level=3)
        elif line.startswith('- '):  # Bullet points
            paragraph = doc.add_paragraph(style='ListBullet')
            process_bold_text(line[2:], paragraph)
        elif line.strip() == '---':  # Horizontal line
            doc.add_paragraph().add_run().add_break()
        else:
            paragraph = doc.add_paragraph()
            process_bold_text(line, paragraph)
    
    doc.save('CopilotX_Guide.docx')
    print("Document saved as 'CopilotX_Guide.docx'.")

def process_bold_text(text, paragraph):
    while '**' in text:
        start_bold = text.find('**')
        end_bold = text.find('**', start_bold + 2)
        if start_bold != -1 and end_bold != -1:
            paragraph.add_run(text[:start_bold])
            paragraph.add_run(text[start_bold+2:end_bold]).bold = True
            text = text[end_bold+2:]
        else:
            break
    paragraph.add_run(text)

In [27]:
markdown_to_docx(response_content)

Document saved as 'CopilotX_Guide.docx'.
